# Spatial Features - finding imagery

Spatial features (spfeas) is a landcover classification tool focused on leveraging imagery textures for classifying landcover. It has been used in the World Bank for both quantifying informal housing and estimating poverty

Running spatial features is a 5 part process:

0. Setting up gbdx for spatial features
1. **Finding imagery**
2. Runnning spfeas
3. Downloading results from AWS
4. Stacking results
5. Running classification

### Finding appropriate imagery
Finding appropriate imagery for the analysis is a two part process. First, this script will search for imagery and generate a CSV of imagery options. Second, the csv and the AOI need to be compared in order to select the best coverage overlay (QGIS is best for this).

In [2]:
# Define input datasets
#AOI = r"C:\Work\NPL_NYU\sample_3apr\sampleShapefile_3apr\sample.shp"
AOI = r"C:\temp\BDI_Imagery_Keith\BEN_AOI.shp"
imageryCSV = AOI.replace(".shp", "_imagerySearch.csv")
### I don't know why these are required for the imagery search - I don't think they do anything, but it works, so just leave it
outFolder = "C:/temp"
searchName = "FUBAR"

cutOffDate = "1-Jan-05"  #earliest acceptable date
optimalDate = "30-Apr-19" #best date
maxAcceptableCloud = 100
curoffNadir = 90

In [3]:
import sys, os, inspect, importlib
import pandas as pd
import geopandas as gpd

from gbdxtools import Interface
from gbdxtools import CatalogImage
from shapely.geometry import box

cmd_folder = os.path.dirname(os.getcwd())
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc
from GOST_GBDx_Tools import imagery_search

#In order for the interface to be properly authenticated, follow instructions here:
#   http://gbdxtools.readthedocs.io/en/latest/user_guide.html
#   For Ben, the .gbdx-config file belongs in C:\Users\WB411133 (CAUSE no one else qill f%*$&ing tell you that)
gbdx = Interface()
gbdx.s3.info
curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)
importlib.reload(imagery_search)

KeyError: 'access_token'

In [11]:
import configparser, requests

gbdxConfigFile = os.path.join(os.path.expanduser('~'), ".gbdx-config")
config = configparser.RawConfigParser()  
''' This part doesn't work in 3.6
with open(gbdxConfigFile) as f:
    sample_config = f.read()
config.readfp(io.BytesIO(sample_config))
'''
config.read(gbdxConfigFile)
username = config.get('gbdx','user_name')
password = config.get('gbdx','user_password')
client_id = gbdx.s3.info['S3_access_key']
secret_key = gbdx.s3.info['S3_secret_key']

url = 'https://geobigdata.io/auth/v1/oauth/token/'
headers = {"Authorization": "Basic %s" % client_id, "Content-Type": "application/x-www-form-urlencoded"}
params = {"grant_type": "password", "username": username, "password": password }
resultsBearer = requests.post(url, headers=headers, data=params)
resultsBearer.json()



{'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6Ik1Ea3hPREE1UTBFeFJUTXpOek01UlVSRE5qWTRRelpHT1ROR1FUWTBNMFJHTnpjMFEwTTFSZyJ9.eyJodHRwczovL2dlb2JpZ2RhdGEuaW8vYWNjb3VudF9sZXZlbCI6ImJhc2ljIiwiaHR0cHM6Ly9nZW9iaWdkYXRhLmlvL2lkIjoiMGVmYjg0MzAtMmQwNi00MmFmLThmNTUtODdhZmVmNTA4ZWE4IiwiaHR0cHM6Ly9nZW9iaWdkYXRhLmlvL2FjY291bnRfaWQiOiIxYzA4MGU5Yy0wMmNjLTRlMmUtYThhMi1iZjA1YjgzNjllZWUiLCJodHRwczovL2dlb2JpZ2RhdGEuaW8vcm9sZXMiOlsidXNlciJdLCJodHRwczovL2dlb2JpZ2RhdGEuaW8vZW1haWwiOiJic3Rld2FydEB3b3JsZGJhbmsub3JnIiwiaXNzIjoiaHR0cHM6Ly9kaWdpdGFsZ2xvYmUtcHJvZHVjdGlvbi5hdXRoMC5jb20vIiwic3ViIjoiYXV0aDB8Z2JkeHwyMzc3MyIsImF1ZCI6WyJnZW9iaWdkYXRhLmlvIiwiaHR0cHM6Ly9kaWdpdGFsZ2xvYmUtcHJvZHVjdGlvbi5hdXRoMC5jb20vdXNlcmluZm8iXSwiaWF0IjoxNTYwNzU3NDYxLCJleHAiOjE1NjEzNjIyNjEsImF6cCI6ImRieFU1Y1pka08wU0hUbXNoRkNXbkk4OTR2eFExTmJ6Iiwic2NvcGUiOiJvcGVuaWQgZW1haWwgb2ZmbGluZV9hY2Nlc3MiLCJndHkiOiJwYXNzd29yZCJ9.wDGAEeSV9GrgSnxJdlm2wh5AEq6Hf4NoQf9YuNXTyrDGOrCw2byb1ENu6-wblF_KyXrkALO5KyM_nS2rTA_WT4u7EHzWrttELMrB0O6nBoli

{'message': 'Unauthorized'}

In [3]:
#Read in viirsExtents if necessary
from shapely.wkt import loads
viirsExtents = r"C:\Work\SouthSudan\VIIRS_2015_extents.csv"
allDissNamed = pd.read_csv(viirsExtents)
allDissNamed['geometry'] = allDissNamed['geometry'].apply(lambda x: loads(x))
allDissNamed = gpd.GeoDataFrame(allDissNamed, geometry='geometry')
allDissNamed.crs = {'init': u'epsg:4326'}
inS = allDissNamed
imageryCSV = viirsExtents.replace(".csv", "_imagerySearch.csv")
#inS = gpd.read_file(AOI)
#inS = inS.to_crs({'init': u'epsg:4326'})
#curWKT = inS['geometry']

In [ ]:
for idx, row in inS.iterrows():
    imageryCSV = AOI.replace(".shp", "_imagerySearch_%s.csv" % idx)
    curWKT = row['geometry']
    curRes = imagery_search.searchForImages(gbdx, curWKT, outFolder, searchName, 
                    cutoff_date=cutOffDate, optimal_date=optimalDate,
                    cutoff_cloud_cover = maxAcceptableCloud, cutoff_nadir = curoffNadir,imageType='*')
    try:
        final = final.append(curRes)
    except:
        final = curRes

In [ ]:
imageryCSV = AOI.replace(".shp", "_imagerySearch.csv")
imageryCSV = viirsExtents.replace(".csv", "_imagerySearch.csv")
final2 = final[~final.duplicated(['ID'])]
final2.to_csv(imageryCSV)

In [ ]:
final2 = pd.read_csv(imageryCSV)


In [15]:
from shapely.wkt import loads
final2['geometry'] = final2['Full_scene_WKT'].apply(loads)
final2 = gpd.GeoDataFrame(final2, geometry='geometry')
final2.crs = {'init': 'epsg:4326'}

In [18]:


def getImages(x, inD):
    ''' kind of like a select by intersection.
    
    INPUT
    x [shapely geometry]
    inD [geopandas dataframe]
    RETURNS
    [geopandas dataframe]    
    '''    
    return(inD[inD.intersects(x)])
    
xx = inS['geometry'].apply(lambda x: getImages(x, final2))

In [37]:
#Prepare list of images that would work for LULC task
allVals = []
for x in xx:
    # limit to Worldview 2 and Worldview3
    x = x[x['ImageBands'] == "PAN_MS1_MS2"]
    # Eliminate images with low overlap
    x = x[x['Overlap_%'] > 25]
    x = x.sort_values('Date')
    oldest = x.iloc[0]
    newest = x.iloc[x.shape[0]-1]
    allVals.append([oldest['ID'], oldest['Date'], newest['ID'], newest['Date']])
selectImages = pd.DataFrame(allVals, columns=['ID_old', 'Date_old', 'ID_new', 'Date_new'])
allImages = selectImages['ID_old'] + selectImages['ID_new']

In [40]:
inSjoin = inS.join(selectImages)
allImages = selectImages['ID_old'] + selectImages['ID_new']

In [46]:
allImages = {}
for oldI in selectImages['ID_new']:#selectImages['ID_old']:
    try:
        curImg = CatalogImage(oldI)
        allImages[oldI] = curImg
    except:
        gbdx.ordering.status(gbdx.ordering.order(oldI))

for oldI in selectImages['ID_old']:
    try:
        curImg = CatalogImage(oldI)
        allImages[oldI] = curImg
    except:
        gbdx.ordering.status(gbdx.ordering.order(oldI))

30

dict_keys(['103001008D773700', '10300100863EFD00', '103001008D6BD100', '104001003F5F2200', '104001004D887A00', '103001008D790500', '103001005497A300', '1030010009114900', '103001008DBE7700', '1030010090117200', '1030010077B27800', '10400100499F9800', '103001000A6D8800', '103001009180B900', '103001008DC6BB00', '104001004B7F4200', '1040010047AFEA00', '1040010049C94E00', '10300100919C0100', '10300100928F9700', '1030010091BE2700', '103001000762F500', '1040010049690500', '103001007856A100', '103001000B912F00', '1030010011C37B00', '104001004A78A100', '103001008E1EB400', '103001008D9BA200', '103001008F7ABF00'])